In [98]:
import pandas as pd
import numpy as np

In [99]:
data = pd.read_csv('DATA/Train.csv')

In [100]:
data.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [101]:
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
data['Category'].unique()

array(['business', 'tech', 'politics', 'sport', 'entertainment'],
      dtype=object)

In [102]:
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")
import re

# Loading stopwords from nltk library
stop_words = set(stopwords.words('english'))
# Function for text preprocessing
def txt_preprocessing(total_text, index, column, df):
    if type(total_text) is not int:
        string = ""
        # Replace every special character with space
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', total_text)
        # Remove multiple spaces
        total_text = re.sub('\s+',' ', total_text)
        # Converting to lowercase
        total_text = total_text.lower()
        
        for word in total_text.split():
        # If word is not a stopword then retain that word from the data
            if not word in stop_words:
                string += word + " "
        df[column][index] = string

In [103]:
for index, row in data.iterrows():
    if type(row['Text']) is str:
        txt_preprocessing(row['Text'], index, 'Text', data)

data.head()

,ArticleId,Text,Category
0,1833,worldcom ex boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens maj...,business
3,1976,lifestyle governs mobile choice faster better ...,tech
4,917,enron bosses 168m payout eighteen former enron...,business


In [115]:
ind = list(data.index)
# np.random.shuffle(ind)

train_len = int(data.shape[0]*0.1)
train_ind = ind[:train_len]
training_data = data.iloc[train_ind,:]
# training_data.head()

# test_ind = ind[train_len:]
# testing_data = data.iloc[test_ind,:]
test_ind = ind[150:180]
testing_data = data.iloc[test_ind,:]
#testing_data.head()

print('Training_data size -> {}'.format(training_data.shape))
print('Testing_data size -> {}'.format(testing_data.shape))

# assert data.shape[0] ==  len(train_ind)+ len(test_ind), 'Not equal distribution'

Training_data size -> (149, 3)
Testing_data size -> (30, 3)


In [122]:
class NB:
    def __init__(self, target, dataframe):
        self.df = dataframe
        # Target/Category Column
        self.c_n = target
        # Column Names
        self.cols = list(self.df.columns)
        self.cols.remove(self.c_n)
        
        self.store = {}
        self.likelihood_for_all_()
        
    def likelihood_cal(self, x, y, z):
        """ 
        x -> Column Name (String)
        y -> Column Value (String)
        z -> Class value (String)
        c_n -> Class Name (Target)
        
        Returns -> P(x = y | c_n = z)
        """
        df = self.df
        
        if x not in self.cols:
            raise KeyError("Feature(column) not present in the Training Dataset")
        
        res =  len(df[(df[x] == y) & (df[self.c_n] == z)]) /len(df[df[self.c_n] == z])
        
        if res == 0.0:
            return 1/(len(df[df[self.c_n] == z]) + len(df[x].unique()))
        
        return res
    
    def likelihood_for_all_(self):     
        df = self.df
        
        dict1 = {}
        for x in self.cols:
            dict2 = {}
            for y in df[x].unique():
                dict3 = {}
                for z in df[self.c_n].unique():
                    #print('P({}="{}"|{}="{}") = {}'.format(x,y,self.c_n,z,self.likelihood_cal(x, y, z)))
                    dict3[z] = self.likelihood_cal(x, y, z)
                dict2[y] = dict3
            dict1[x] = dict2
        
        self.store = dict1
    
    def likelihood_expr(self, class_val, expr):
        val = 1  
        
        for k,v in expr:
            try:
                store_val = self.store[k][v][class_val]
            except:
                store_val = self.likelihood_cal(k,v,class_val)
                
            val *= store_val
                                         
        return val
    
    def prior(self, class_val):
        df = self.df
        return len(df[df[self.c_n] == class_val])/df.shape[0]
    
    def predict(self, X):
        df = self.df
        
        if type(X) == pd.core.series.Series:
            values_list = [list(X.items())]
            
        elif type(X) == pd.core.frame.DataFrame:
            values_list = [list(y.items()) for x,y in X.iterrows()]
            
        else:
            raise TypeError('{} is not supported type'.format(type(X)))
            
        
        predictions_list = []
        for values in values_list:
            likelihood_priors = {}
            for class_val in df[self.c_n].unique():
                likelihood_priors[class_val] = self.prior(class_val)*self.likelihood_expr(class_val,values)
            #print(likelihood_priors)
            
            normalizing_prob = np.sum([x for x in likelihood_priors.values()])
            probabilities = [(y/normalizing_prob,x) for x,y in likelihood_priors.items()]
           
            
            if len(probabilities) == 2:
                # For 2 Class Predictions
                max_prob = max(probabilities)[1]
                predictions_list.append(max_prob)
            
            else:
                # For Mulit Class Predictions
                exp_1 = [np.exp(x) for x,y in probabilities]
                exp_2 = np.sum(exp_1)
                softmax = exp_1/exp_2
                #print(softmax)
                class_names = [y for x,y in probabilities]
                softmax_values = [(x,y) for x,y in zip(softmax,class_names)]
                #print(softmax_values)
                max_prob = max(softmax_values)[1]
                predictions_list.append(max_prob)
        
        print(probabilities)
        return predictions_list
    
    def accuracy_score(self, X, Y):
        assert len(X) == len(Y), 'Given values are not equal in size'
        
        total_matching_values = [x == y for x,y in zip(X,Y)]
        return (np.sum(total_matching_values)/len(total_matching_values))*100
    
    def calculate_confusion_matrix(self, X, Y):
        df = self.df
        
        unique_class_values = df[self.c_n].unique()
        decimal_class_values = list(range(len(unique_class_values)))
        numerical = {x:y for x,y in zip(unique_class_values, decimal_class_values)}
        
        x = [numerical[x] for x in X]
        y = [numerical[y] for y in Y]
        
        
        n = len(decimal_class_values)
        confusion_matrix = np.zeros((n,n))
        
        for i,j in zip(x,y):
            if i == j:
                confusion_matrix[i][i] += 1
            elif i != j:
                confusion_matrix[i][j] += 1
        
        return confusion_matrix
            
    
    def precision_score(self, X, Y):
        assert len(X) == len(Y), 'Given values are not equal in size'
        
        confusion_matrix = self.calculate_confusion_matrix(X,Y)
        tp = confusion_matrix[0][0]
        fp = confusion_matrix[1][0]
        
        return tp / (tp+fp)
    
    def recall_score(self, X, Y):
        assert len(X) == len(Y), 'Given values are not equal in size'
        
        confusion_matrix = self.calculate_confusion_matrix(X,Y)
        tp = confusion_matrix[0][0]
        fn = confusion_matrix[0][1]
        
        return tp / (tp+fn)

In [123]:
genx = NB(target='Category',dataframe=training_data)

In [124]:
y_test = list(testing_data.iloc[0:20,2])

y_pred = genx.predict(testing_data.iloc[0:20, 0:1])
print(y_test)
print(y_pred)
print('Accuracy Score -> {} %'.format(round(genx.accuracy_score(y_test,y_pred),3)))
# print('Precison Score -> {}'.format(round(genx.precision_score(y_test,y_pred),3)))
# print('Recall Score -> {}'.format(round(genx.recall_score(y_test,y_pred),3)))

[(0.25186479181230315, 'business'), (0.1862566554835738, 'tech'), (0.15620224605423388, 'politics'), (0.2619937672455105, 'sport'), (0.14368253940437867, 'entertainment')]
['sport', 'sport', 'politics', 'politics', 'tech', 'business', 'business', 'entertainment', 'tech', 'entertainment', 'politics', 'tech', 'entertainment', 'sport', 'business', 'politics', 'entertainment', 'sport', 'politics', 'sport']
['sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport', 'sport']
Accuracy Score -> 25.0 %
